In [1]:
import pandas as pd
import numpy as np
import gmaps
import gmaps.geojson_geometries
from matplotlib.cm import viridis
from matplotlib.colors import to_hex

In [2]:
state_brewery_df = pd.read_csv("resources/state_table_for_per_capita_analysis.csv", index_col = 0)
state_pops_df = pd.read_csv("resources/population_data_2016_2018.csv", index_col = 0)

In [3]:
state_brewery_df.head()

,STATE,Mean # Total Income Returns,Mean Total Income,Mean # Returns with Salaries and Wages,Mean Salaries and Wages,Brewery Count,state_name
0,AK,2588.988095,183958.494048,2198.571429,127301.089286,41,Alaska
1,AL,3469.950739,200435.201970,2918.587849,140634.338259,42,Alabama
2,AR,2655.411090,163030.388145,2250.516252,105676.304015,39,Arkansas
3,AZ,8370.093897,531552.845070,6929.859155,361805.767606,118,Arizona
4,CA,11992.881944,978355.903274,9985.466270,667623.095238,847,California


In [4]:
state_pops_df.head()

,Alpha,FIPS,POPESTIMATE2016,POPESTIMATE2018
0,AL,1,4864745,4887871
1,AK,2,741504,737438
2,AZ,4,6945452,7171646
3,AR,5,2990410,3013825
4,CA,6,39209127,39557045


In [5]:
state_pops_df.rename(columns = {
    "Alpha": "STATE",
    "POPESTIMATE2016": "2016_Pop",
    "POPESTIMATE2018": "2018_Pop"
}, inplace = True)

In [6]:
per_capita_df = state_brewery_df.merge(state_pops_df)
per_capita_df.head()

,STATE,Mean # Total Income Returns,Mean Total Income,Mean # Returns with Salaries and Wages,Mean Salaries and Wages,Brewery Count,state_name,FIPS,2016_Pop,2018_Pop
0,AK,2588.988095,183958.494048,2198.571429,127301.089286,41,Alaska,2,741504,737438
1,AL,3469.950739,200435.201970,2918.587849,140634.338259,42,Alabama,1,4864745,4887871
2,AR,2655.411090,163030.388145,2250.516252,105676.304015,39,Arkansas,5,2990410,3013825
3,AZ,8370.093897,531552.845070,6929.859155,361805.767606,118,Arizona,4,6945452,7171646
4,CA,11992.881944,978355.903274,9985.466270,667623.095238,847,California,6,39209127,39557045


In [7]:
per_capita_df.dtypes

STATE                                      object
Mean # Total Income Returns               float64
Mean Total Income                         float64
Mean # Returns with Salaries and Wages    float64
Mean Salaries and Wages                   float64
Brewery Count                               int64
state_name                                 object
FIPS                                        int64
2016_Pop                                    int64
2018_Pop                                    int64
dtype: object

In [8]:
# Create Per Capita Columns
per_capita_df["breweries_per_capita_2016"] = per_capita_df["Brewery Count"]/per_capita_df["2016_Pop"]
per_capita_df["breweries_per_capita_2018"] = per_capita_df["Brewery Count"]/per_capita_df["2018_Pop"]
per_capita_df.head()

,STATE,Mean # Total Income Returns,Mean Total Income,Mean # Returns with Salaries and Wages,Mean Salaries and Wages,Brewery Count,state_name,FIPS,2016_Pop,2018_Pop,breweries_per_capita_2016,breweries_per_capita_2018
0,AK,2588.988095,183958.494048,2198.571429,127301.089286,41,Alaska,2,741504,737438,0.000055,0.000056
1,AL,3469.950739,200435.201970,2918.587849,140634.338259,42,Alabama,1,4864745,4887871,0.000009,0.000009
2,AR,2655.411090,163030.388145,2250.516252,105676.304015,39,Arkansas,5,2990410,3013825,0.000013,0.000013
3,AZ,8370.093897,531552.845070,6929.859155,361805.767606,118,Arizona,4,6945452,7171646,0.000017,0.000016
4,CA,11992.881944,978355.903274,9985.466270,667623.095238,847,California,6,39209127,39557045,0.000022,0.000021


In [9]:
max(per_capita_df["breweries_per_capita_2016"])

0.00010101917119382211

In [10]:
max(per_capita_df["breweries_per_capita_2018"])

0.00010059093180733164

## Mapping

In [18]:
# Create dictionary of state names and breweries per capita to assign colors to geos

brewer_per_cap_dict_18 = dict(zip(per_capita_df.state_name, per_capita_df.breweries_per_capita_2018))
brewer_per_cap_dict_16 = dict(zip(per_capita_df.state_name, per_capita_df.breweries_per_capita_2016))

In [12]:
%matplotlib inline
states_geojson = gmaps.geojson_geometries.load_geometry("us-states")
fig = gmaps.figure()
states_layer = gmaps.geojson_layer(states_geojson)
fig.add_layer(states_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### 2018 Breweries per Capita

In [13]:
# Create range

min_brewers = min(brewer_per_cap_dict_18.values())
max_brewers = max(brewer_per_cap_dict_18.values())
brewers_range = max_brewers - min_brewers

In [14]:
def calculate_color(brewers_count):
    # Convert the count to a color
    # Convert count to a number between 0 and 1
    normalized_brewers = (brewers_count - min_brewers) / brewers_range
    
    # Invert it so higher counts get a darker color
    inverse_brewers = 1.0 - normalized_brewers
    
    # Transform counts to a matplotlib color
    mpl_color = viridis(inverse_brewers)
    
    # Transform the matplotlib colors to CSS colors to apply to gmaps
    gmaps_color = to_hex(mpl_color, keep_alpha=False)
    
    return gmaps_color

In [15]:
colors = []

for feature in states_geojson["features"]:
    state_name = feature["properties"]["NAME"]
    try:
        brew_count = brewer_per_cap_dict_18[state_name]
        color = calculate_color(brew_count)
    except KeyError:
        # No count available, return a default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

In [16]:
%matplotlib inline

fig1 = gmaps.figure()
states_layer = gmaps.geojson_layer(states_geojson, \
                                   fill_color = colors, \
                                   stroke_color = colors, \
                                   fill_opacity = 0.8)
fig1.add_layer(states_layer)
fig1

Figure(layout=FigureLayout(height='420px'))

### Breweries per Capita 2016

In [19]:
# Create range

min_brewers = min(brewer_per_cap_dict_16.values())
max_brewers = max(brewer_per_cap_dict_16.values())
brewers_range = max_brewers - min_brewers

In [20]:
def calculate_color(brewers_count):
    # Convert the count to a color
    # Convert count to a number between 0 and 1
    normalized_brewers = (brewers_count - min_brewers) / brewers_range
    
    # Invert it so higher counts get a darker color
    inverse_brewers = 1.0 - normalized_brewers
    
    # Transform counts to a matplotlib color
    mpl_color = viridis(inverse_brewers)
    
    # Transform the matplotlib colors to CSS colors to apply to gmaps
    gmaps_color = to_hex(mpl_color, keep_alpha=False)
    
    return gmaps_color

In [22]:
colors = []

for feature in states_geojson["features"]:
    state_name = feature["properties"]["NAME"]
    try:
        brew_count = brewer_per_cap_dict_16[state_name]
        color = calculate_color(brew_count)
    except KeyError:
        # No count available, return a default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

In [23]:
%matplotlib inline

fig1 = gmaps.figure()
states_layer = gmaps.geojson_layer(states_geojson, \
                                   fill_color = colors, \
                                   stroke_color = colors, \
                                   fill_opacity = 0.8)
fig1.add_layer(states_layer)
fig1

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Unsurprisingly, not a noticeable difference between 2016 and '18.